## 4-Qubit 2-Layer Quantum Born Machine (QBM) for Traffic Simulation

A Quantum Born Machine (QBM) is a generative quantum model that learns to reproduce a target probability distribution using the Born rule.
![4-qubit-2-layer-QBM](images/4-qubit-2-layer-QBM.png)
In this lab, the target data represents a typical urban traffic pattern with two peaks: morning and late afternoon, corresponding to rush-hour traffic flows. The QBM uses a parameterized 4-qubit quantum circuit to approximate this distribution.

### Loss Function
The QBM minimizes the Jensen–Shannon (JS) divergence, which symmetrizes the Kullback–Leibler divergence:

$$
JS(P \parallel Q) = \tfrac{1}{2} KL(P \parallel M) + \tfrac{1}{2} KL(Q \parallel M)
$$

where  
- $P$ is the model distribution,  
- $Q$ is the target distribution, and  
- $M = \tfrac{1}{2}(P+Q)$ is the midpoint distribution.

The complete formula is:

$$
JS(P \parallel Q) =
\tfrac{1}{2} \sum_x P(x) \cdot \log \frac{P(x)}{M(x)} \;+\;
\tfrac{1}{2} \sum_x Q(x) \cdot \log \frac{Q(x)}{M(x)}
$$

where

$$
M(x) = \tfrac{1}{2} \big(P(x) + Q(x)\big)
$$

This provides a bounded, symmetric measure of dissimilarity between the model and target.

## SPSA Optimization
The parameters of the circuit are updated using SPSA (Simultaneous Perturbation Stochastic Approximation), a gradient-free optimization algorithm. At each step, SPSA perturbs all parameters with random noise, estimates the gradient using just two function evaluations, and adjusts parameters accordingly. This makes SPSA particularly efficient for noisy quantum simulations.

#### SPSA Algorithm Steps
1. Initialization:  
   - Copy the initial parameters $\theta_0$ into a working variable $\theta$.  
   - Create an empty list `losses` to record loss values over iterations.

2. Iterative Optimization Loop (repeat for $max\_iter$ steps):
   - Generate random perturbation $\delta$:  
     Create a perturbation vector $\delta$, where each component is randomly chosen as $+1$ or $-1$.  
   - Perturb parameters:  
     Create two perturbed versions of $\theta$:  
     $$
     \theta^+ = \theta + c \delta, \quad \theta^- = \theta - c \delta
     $$
   - Evaluate loss at perturbed points:  
     Compute the loss at both perturbed parameter sets:  
     $$
     L^+ = loss\_fn(\theta^+), \quad L^- = loss\_fn(\theta^-)
     $$
   - Estimate the gradient:  
     Use the finite-difference approximation to estimate the gradient:  
     $$
     grad\_estimate = \frac{L^+ - L^-}{2c} \, \delta
     $$
     This requires only two loss evaluations, regardless of the size of $\theta$.  
   - Update parameters:  
     Apply the estimated gradient to update the parameters:  
     $$
     \theta \leftarrow \theta - \alpha \cdot grad\_estimate
     $$
   - Evaluate and store current loss:  
     Compute the loss at the new $\theta$ and append it to the list `losses`.

3. Return Results:  
   Return the final optimized parameter vector and the full list of loss values for convergence plotting.

---

### Task
Build a 4-qubit, 2-layer Quantum Born Machine (QBM) that learns the given traffic probability distribution.  
The circuit is trained by minimizing the Jensen–Shannon (JS) divergence between the model output and the target distribution, using Simultaneous Perturbation Stochastic Approximation (SPSA) as the optimization routine.


### Expected Output
- A plot showing the JS loss convergence over SPSA iterations.  
- The final learned distribution from the QBM compared side-by-side with the target urban traffic distribution.  
- A quantum circuit diagram of the optimized QBM.


## Experimentation
- Vary the number of shots in sampling to explore the effect of measurement noise.  
- Increase the number of layers or change entanglement patterns to test circuit expressiveness.  
- Compare SPSA with other optimizers (e.g., COBYLA, gradient descent) and observe convergence behavior.  
- Replace the JS divergence with other loss functions.

---

### References
- Simultaneous Perturbation Stochastic Approximation (SPSA):  
  Spall, J. C. (1992). *Multivariate stochastic approximation using a simultaneous perturbation gradient approximation.* IEEE Transactions on Automatic Control, 37(3), 332–341.  

- Jensen–Shannon (JS) Divergence:  
  Lin, J. (1991). *Divergence measures based on the Shannon entropy.* IEEE Transactions on Information Theory, 37(1), 145–151.  


In [ ]:
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit_aer import AerSimulator


# -- Target distribution to learn --
p_dist = [0.03, 0.05, 0.07, 0.10, 0.12, 0.08, 0.05, 0.04,
          0.04, 0.06, 0.10, 0.08, 0.05, 0.05, 0.04, 0.04] 
p_target = {format(i, '04b'): p for i, p in enumerate(p_dist)}

# -- Create QBM circuit with 8 parameters --
def create_qbm_circuit(theta):
    qc = QuantumCircuit(4)
    
    # First Ry layer
    for i in range(4):
        qc.ry(theta[i], i)
        
    # First CX layer
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    qc.cx(3, 0)
    qc.barrier()

    # Second Ry layer
    for i in range(4):
        qc.ry(theta[4 + i], i)

    # Second CX layer (entangle differently if needed)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    qc.cx(3, 0)

    qc.measure_all()
    return qc
    
# -- Sample from QBM circuit using AerSimulator --
def sample_qbm(theta, shots=1000):
    qc = create_qbm_circuit(theta)
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    total = sum(counts.values())
    p_model = {bit: count / total for bit, count in counts.items()}
    return p_model

# -- Jensen–Shannon (JS) divergence loss function  --
def js_loss(theta):
    p_model = sample_qbm(theta)
    eps = 1e-10  # small value to avoid log(0)
    js = 0.0
    for k in p_target:
        p = p_model.get(k, eps)
        q = p_target[k]
        m = 0.5 * (p + q)
        if p > 0:
            js += 0.5 * p * np.log(p / m)
        if q > 0:
            js += 0.5 * q * np.log(q / m)
    return js

# -- Simultaneous Perturbation Stochastic Approximation  (SPSA) optimization  --
def spsa(loss_fn, theta0, alpha=0.2, c=0.1, max_iter=200):
    theta = np.copy(theta0)
    n = len(theta)
    losses = []

    for step in range(max_iter):
        delta = 2 * np.random.randint(0, 2, size=n) - 1
        theta_plus = theta + c * delta
        theta_minus = theta - c * delta
        loss_plus = loss_fn(theta_plus)
        loss_minus = loss_fn(theta_minus)
        grad_estimate = (loss_plus - loss_minus) / (2 * c) * delta
        theta = theta - alpha * grad_estimate
        current_loss = loss_fn(theta)
        losses.append(current_loss)
    return theta, losses

# -----------------------------------------------
#                main program
# -----------------------------------------------

# -- Run SPSA optimization --
theta0 = np.random.uniform(0, 2 * np.pi, 8)
theta_opt, losses = spsa(js_loss, theta0, max_iter=250)

# -- Plot convergence of loss --
plt.figure(figsize=(8, 4))
plt.plot(losses, marker='.', linestyle='-')
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("QBM Loss Convergence Over Iterations")
plt.grid(True)
plt.tight_layout()
plt.show()

# -- Final learned distribution --
p_qbm = sample_qbm(theta_opt)
qc = create_qbm_circuit(theta_opt)
display(circuit_drawer(qc, style="bw", output="mpl"))

# -- Plot learned vs. target distribution --
bitstrings = sorted(set(p_target.keys()).union(p_qbm.keys()))
target_probs = [p_target.get(b, 0) for b in bitstrings]
qbm_probs = [p_qbm.get(b, 0) for b in bitstrings]

x = np.arange(len(bitstrings))
bar_width = 0.35

plt.figure(figsize=(10, 4))
plt.bar(x - bar_width / 2, target_probs, width=bar_width, label="Target")
plt.bar(x + bar_width / 2, qbm_probs, width=bar_width, label="QBM Output")
plt.xticks(x, bitstrings, rotation=45, ha='right')
plt.ylabel("Probability")
plt.title("Target vs QBM Output Distribution")
plt.legend()
plt.tight_layout()
plt.show()
